In [76]:
import numpy as np
import torch
import opt_einsum
import itertools

import process_matrices
import utils

from tqdm import tqdm

In [77]:
q_s = 1
q_c = 6

In [78]:
# Generalized Amplitude Damping Krauss Operators
N = 0
g = 0.1

K_1 = torch.tensor([[np.sqrt(1 - N), 0],[0, np.sqrt(1 - N) * np.sqrt(1 - g)]])
K_2 = torch.tensor([[0,np.sqrt(g*(1-N))],[0,0]])
K_3 = torch.tensor([[np.sqrt(N)*np.sqrt(1-g), 0],[0,np.sqrt(N)]])
K_4 = torch.tensor([[0,0],[np.sqrt(g * N), 0]])
K = [K_1, K_2, K_3, K_4]

In [79]:
pms = process_matrices.ProcessMatrices(q_s, q_c, K)

In [80]:
optimizer = torch.optim.Adam(pms.parameters(), lr = 1e-5)

In [81]:
regularization = 10

pbar = tqdm(range(100000))
for epoch in pbar:
    X_C, X_E, X_R = pms()
    f_avg = utils.avg_fidelity(X_C, X_E, X_R)
    X_C_identity = utils.sums_to_identity(X_C)
    X_R_identity = utils.sums_to_identity(X_R)


    l = -f_avg + regularization * (X_C_identity + X_R_identity)
    l.backward()
    optimizer.step()

    pbar.set_description(f"Total Loss : {l:.2f}, Avg Fidelity : {f_avg:.2f}, X_C sums to identity : {X_C_identity:.2f}, X_R sums to identity : {X_R_identity:.2f}")

Total Loss : 659.28, Avg Fidelity : 0.00, X_C sums to identity : 1.96, X_R sums to identity : 63.96:   0%|          | 57/100000 [00:16<8:00:59,  3.46it/s]


KeyboardInterrupt: 